# Generate data 
Use Grid2Op for data generation with specific chronics and agent


In [1]:
import os
import sys
import grid2op
import copy

import numpy as np
import plotly.graph_objects as go

from pprint import pprint
from tqdm.notebook import tqdm

from grid2op.Agent import PowerLineSwitch, BaseAgent, DoNothing, DoNothingAgent, GreedyAgent, TopologyGreedy
from grid2op.Reward import L2RPNReward, L2RPNSandBoxScore
from grid2op.Runner import Runner
from grid2op.Parameters import Parameters

from lightsim2grid import LightSimBackend
backend = LightSimBackend()

max_iter = 1300

In [2]:
param = Parameters()
param.MAX_SUB_CHANGED = 99999
param.MAX_LINE_STATUS_CHANGED = 9999

# Parameters to define for overflow discarding
param.NB_TIMESTEP_COOLDOWN_SUB = 0
param.NB_TIMESTEP_COOLDOWN_LINE = 0
param.NB_TIMESTEP_RECONNECTION = 0
param.NB_TIMESTEP_OVERFLOW_ALLOWED = 9999
param.NO_OVERFLOW_DISCONNECTION = True

#param.NO_OVERFLOW_DISCONNECTION = True
#param.HARD_OVERFLOW_THRESHOLD = 9999

In [3]:
scenario = "rte_case14_realistic"
agent_name = "DoNothingAgent"
agent_class = DoNothingAgent

In [4]:
path_agents = scenario

In [5]:
scoring_function = L2RPNReward
env = grid2op.make(scenario, backend=backend, param=param)
env.set_thermal_limit([1e20 for _ in range(env.n_line)])
env.verbose = True
pprint(env.get_params_for_runner())


if not os.path.exists(path_agents):
    os.mkdir(path_agents)

# make a runner for this agent
path_agent = os.path.join(path_agents, agent_name)

runner = Runner(**env.get_params_for_runner(),
                agentClass=agent_class
                )
res = runner.run(path_save=path_agent, nb_episode=100, 
                max_iter=max_iter,
                pbar=tqdm)
print("The results for the evaluated agent are:")
for _, chron_id, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics with id {}\n".format(chron_id)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

{'actionClass': <class 'grid2op.Action.TopologyAndDispatchAction.TopologyAndDispatchAction'>,
 'backendClass': <class 'lightsim2grid.LightSimBackend.LightSimBackend'>,
 'envClass': <class 'grid2op.Environment.Environment.Environment'>,
 'gridStateclass': <class 'grid2op.Chronics.MultiFolder.Multifolder'>,
 'gridStateclass_kwargs': {'gridvalueClass': <class 'grid2op.Chronics.GridStateFromFileWithForecasts.GridStateFromFileWithForecasts'>},
 'grid_layout': {'sub_0': (-280.0, -81.0),
                 'sub_1': (-100.0, -270.0),
                 'sub_10': (79.0, 162.0),
                 'sub_11': (-170.0, 270.0),
                 'sub_12': (-64.0, 270.0),
                 'sub_13': (222.0, 216.0),
                 'sub_2': (366.0, -270.0),
                 'sub_3': (366.0, -54.0),
                 'sub_4': (-64.0, -54.0),
                 'sub_5': (-64.0, 54.0),
                 'sub_6': (450.0, 0.0),
                 'sub_7': (550.0, 0.0),
                 'sub_8': (326.0, 54.0),
         



The results for the evaluated agent are:
	For chronics with id 000
		 - cumulative reward: 1413796.375000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 001
		 - cumulative reward: 1404022.125000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 002
		 - cumulative reward: 1403251.250000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 003
		 - cumulative reward: 1423209.625000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 004
		 - cumulative reward: 1438636.875000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 005
		 - cumulative reward: 1419330.125000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 006
		 - cumulative reward: 1423139.250000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 007
		 - cumulative reward: 1398483.250000
		 - number of time steps completed: 1300 / 1300
	For chronics with id 008
		 - cumulative rew

## Looking at the results and understanding the behaviour of the Agent

Now we can load the actions and observations and put in in a dataframe

In [6]:
from grid2op.Episode import EpisodeData
import pandas as pd 

In [7]:
path_agent = os.path.join(path_agents, agent_name)

observations = []
for episode_path in EpisodeData.list_episode(path_agent): 
    episode = EpisodeData.from_disk(*episode_path)
    
    for o in episode.observations:
        observation = []
        observation += [o.year, 
                        o.month,
                        o.day, 
                        o.hour_of_day, 
                        o.minute_of_hour, 
                        o.day_of_week]
        
        observation += list(o.prod_p)
        observation += list(o.prod_q)
        observation += list(o.prod_v)

        observation += list(o.load_p)
        observation += list(o.load_q)
        observation += list(o.load_v)
        
        observation += list(o.p_or)
        observation += list(o.q_or)
        observation += list(o.v_or)
        observation += list(o.a_or)
        
        observation += list(o.p_ex)
        observation += list(o.q_ex)
        observation += list(o.v_ex)
        observation += list(o.a_ex)

        observation += list(o.topo_vect)
        
        observation += list(o.rho)
        
        observations.append(observation)


In [8]:
topo_vect_columns = ['' for _ in range(len(o.topo_vect))]

for load_i, position_i in enumerate(grid2op.Space.GridObjects.load_pos_topo_vect): 
    topo_vect_columns[position_i]= 'topo_vect_load_' + str(load_i)

for load_i, position_i in enumerate(grid2op.Space.GridObjects.gen_pos_topo_vect): 
    topo_vect_columns[position_i]= 'topo_vect_gen_' + str(load_i)
    
for load_i, position_i in enumerate(grid2op.Space.GridObjects.line_or_pos_topo_vect): 
    topo_vect_columns[position_i]= 'topo_vect_line_or_' + str(load_i)

for load_i, position_i in enumerate(grid2op.Space.GridObjects.line_ex_pos_topo_vect): 
    topo_vect_columns[position_i]= 'topo_vect_line_ex_' + str(load_i)

In [9]:
observations_columns = ["year", 
                       "month", 
                       "day", 
                       "hour_of_day", 
                       "minute_of_hour", 
                       "day_of_week", 

                       ]
observations_columns += ["prod_p_" + gen for gen in o.name_gen]
observations_columns += ["prod_q_" + gen for gen in o.name_gen]
observations_columns += ["prod_v_" + gen for gen in o.name_gen]

observations_columns += ["load_p_" + load for load in o.name_load]
observations_columns += ["load_q_" + load for load in o.name_load]
observations_columns += ["load_v_" + load for load in o.name_load]

observations_columns += ["line_p_or_" + line for line in o.name_line]
observations_columns += ["line_q_or_" + line for line in o.name_line]
observations_columns += ["line_v_or_" + line for line in o.name_line]
observations_columns += ["line_a_or_" + line for line in o.name_line]

observations_columns += ["line_p_ex_" + line for line in o.name_line]
observations_columns += ["line_q_ex_" + line for line in o.name_line]
observations_columns += ["line_v_ex_" + line for line in o.name_line]
observations_columns += ["line_a_ex_" + line for line in o.name_line]

observations_columns += topo_vect_columns

observations_columns += ["line_rho_" + line for line in o.name_line]



observations_df = pd.DataFrame(observations, columns=observations_columns)

In [16]:
observations_df.head()

,year,month,day,hour_of_day,minute_of_hour,day_of_week,prod_p_gen_1_0,prod_p_gen_2_1,prod_p_gen_5_2,prod_p_gen_7_3,...,line_rho_8_9_10,line_rho_8_13_11,line_rho_9_10_12,line_rho_11_12_13,line_rho_12_13_14,line_rho_3_6_15,line_rho_3_8_16,line_rho_4_5_17,line_rho_6_7_18,line_rho_6_8_19
0,2019,1,6,0,0,6,81.300003,80.099998,12.6,0.0,...,9.696737e-19,2.361358e-18,2.635979e-18,6.463157e-19,2.887801e-18,1.243079e-18,6.137629e-19,1.485756e-18,9.178147e-18,7.108563e-18
1,2019,1,6,0,5,6,81.599998,81.099998,12.9,0.0,...,9.488822e-19,2.399917e-18,2.647153e-18,6.745319e-19,2.913341e-18,1.242648e-18,6.142983e-19,1.482842e-18,9.188407e-18,7.128031e-18
2,2019,1,6,0,10,6,81.000000,80.900002,12.8,0.0,...,9.301504e-19,2.418240e-18,2.645691e-18,6.611092e-19,2.942178e-18,1.250759e-18,6.190831e-19,1.495626e-18,9.217493e-18,7.181705e-18
3,2019,1,6,0,15,6,82.199997,79.800003,12.9,0.0,...,9.996127e-19,2.418041e-18,2.635954e-18,6.657896e-19,2.908902e-18,1.231840e-18,6.059953e-19,1.475210e-18,9.190885e-18,7.021873e-18
4,2019,1,6,0,20,6,82.199997,79.599998,13.0,0.0,...,8.986705e-19,2.419013e-18,2.639629e-18,6.698997e-19,2.929139e-18,1.231850e-18,6.063978e-19,1.469349e-18,9.211928e-18,7.037471e-18


In [14]:
data_file_name = scenario + '-' + agent_name
data_file_name

'rte_case14_realistic-DoNothingAgent'

In [15]:
observations_df.to_feather("../" + data_file_name + ".feather")

What are the features in this dataset ? 

In [12]:
['year', #date
 'month', #date
 'day', #date
 'hour_of_day', #date
 'minute_of_hour', #date
 'day_of_week', #date
 'prod_p', # productions
 'prod_q', # productions
 'prod_v', # productions
 'load_p', # loads
 'load_q', # loads
 'load_v', # loads
 'p_or', # lines
 'q_or', # lines
 'v_or', # lines
 'a_or', # lines
 'p_ex', # lines
 'q_ex', # lines
 'v_ex', # lines
 'a_ex', # lines
 'topo_vect', 
 'rho' # line capacity used (current flow / thermal limit) 
]

['year',
 'month',
 'day',
 'hour_of_day',
 'minute_of_hour',
 'day_of_week',
 'prod_p',
 'prod_q',
 'prod_v',
 'load_p',
 'load_q',
 'load_v',
 'p_or',
 'q_or',
 'v_or',
 'a_or',
 'p_ex',
 'q_ex',
 'v_ex',
 'a_ex',
 'topo_vect',
 'rho']